# Парсинг HTML. Beautiful Soup
Оглавление:

Beautiful Soup для сбора данных в HTML
- Установка и начало работы
-  Дерево синтаксического разбора
- Атрибуты тегов
- Навигация по дереву синтаксического разбора

> parent

> contents

> string

> next_sibling и previous_sibling

> next и previous

- Глоссарий
- Дополнительные материалы
- Домашнее задание
- Используемая литература

1. Познакомимся с библиотекой BeautifulSoup.
2. Напишем первый парсер сайта с использованием этой библиотеки.


### Beautiful Soup для сбора данных в HTML
#### Установка и начало работы

Парсить HTML мы будем с применением библиотеки Requests, а обрабатывать полученную структуру — с помощью библиотеки Beautiful Soup. Установим их, введя команды в терминале:


In [ ]:
!pip install bs4

In [ ]:
!pip install requests

Импортируем библиотеки:

In [1]:
import requests
from bs4 import BeautifulSoup as bs

Чтобы посмотреть основные возможности Beautiful Soup, попробуем собрать все книги с первой
страницы известного нам сайта Books to Scrape.

Для начала сделаем GET-запрос на сайт:

In [2]:
from bs4 import BeautifulSoup as bs
import requests
response = requests.get('http://example.com')

Теперь передадим полученный ответ в Beautiful Soup и укажем, какой парсер нам нужен. Создадим объект «суп»:

In [4]:
soup = bs(response.content, 'html.parser')

### Дерево синтаксического разбора
Дерево синтаксического разбора — структуры данных Beautiful Soup, которые создаются по мере синтаксического разбора документа. Объект парсера (экземпляр класса BeautifulSoup) обладает большой глубиной вложенности связанных структур данных, соответствующих структуре документа
XML или HTML. Объект парсера состоит из объектов двух других типов:

● объектов Tag, которые соответствуют тегам, к примеру, `<div>` и `<p>`;

● объекты NavigableString, соответствующие таким строкам, как In stock или Sapiens: A Brief History ....

### Атрибуты тегов
У тегов HTML есть атрибуты: например, у каждого тега `<img>` в приведённом выше примере HTML есть атрибуты class, src и alt. К атрибутам тегов можно обращаться таким же образом, как если бы объект Tag был словарём:

In [6]:
soup.head.parent.name
# 'html'
soup.head.parent.parent.__class__.__name__
# 'BeautifulSoup'
soup.parent == None
# true

True

#### contents
Parent перемещает нас вверх по дереву синтаксического разбора. С помощью contents мы перемещаемся вниз по дереву синтаксического разбора. Сontents — упорядоченный список, состоящий — из объектов Tag и NavigableString, содержащихся в элементе страницы (page element).

#### string
Если у тега есть только один дочерний узел, который будет строкой, этот узел будет доступен через tag.string точно так же, как и через tag.contents[0].

#### next_sibling и previous_sibling
Эти элементы позволяют переходить к следующему или предыдущему элементу на этом же уровне дерева синтаксического разбора. Эти способы перемещения довольно часто используются, когда у вас есть одинаковые элементы с абсолютно одинаковыми атрибутами, и вы точно знаете, что вам нужен именно второй.

#### next и previous
Эти элементы позволяют передвигаться по элементам документа в том порядке, в котором они были обработаны парсером, а не в порядке появления в дереве.

Спарсим информацию о каждой книге с первой страницы. Для начала нам надо понять, где находится каждая книга. Нажимаем вот на этот значок в левом углу нашего инспектора и наводим на книгу. Видим, что вся информация о книге находится в теге article, родителем которого является тег li.
Давайте посмотрим, сколько у нас всего элементов с тегом article на странице, для этого в строку поиска вводим article и нажимаем enter. Поиск показывает 20 элементов, и на самом верху страницы
мы видим, что нам показано 20 элементов. Таким образом, чтобы выбрать все блоки с книгами с первой страницы, нам надо написать следующий код:

In [8]:
books = soup.find_all('article')
books2 = soup.select('article')
print(len(books), len(books2))

0 0


Мы использовали оба метода, и итоговая длина списков у нас получилась одинаковой. Давайте посмотрим на первый элемент списка books — print(books[0]). Как видите, это элемент HTML, то есть, обращаясь к нему, мы можем применять все методы, которые доступны для супа.

Сохраним его в переменную book и протестируем для него получение названия, обложки, цены и наличия. Будем обращаться к элементу book и вызывать для него методы, доступные для супа. Рассмотрим разные способы получения информации.

Первое, что нам надо получить — название. Мы внутри тега article, так что всё, что выше, нас не интересует. Название находится в теге `<а>`, который вложен в тег `<h3>`. Возьмём этот тег за основу и спустимся вниз. Обратите внимание: если текст мы возьмём у тега `<а>`, то получим неполное название, так что нам надо брать атрибут title. Итак:

In [ ]:
title = book.find('h3').find('a')['title']

Теперь найдём цену. Возвращаемся на сайт. Цена находится в теге `<p>` с классом `price_color`, который вложен в тег `<div>`. Внутри этого же тега лежит и информация о наличии книги в магазине. Сперва получим информацию о наличии товара. Мы можем сделать это двумя способами.

Первый — используя метод find:

In [ ]:
instock = book.find('p', attrs={'class':['instock', 'availability']}).text
print(instock)
print(instock.strip())

Метод strip мы используем, чтобы избавиться от лишних пробелов и энтеров у текста.

Второй — используя метод `select_one` и указание неполного класса. Этот метод удобен, когда вы точно знаете, что внутри html находится единственный тег с указанным классом или вам нужен первый тег из множества. Метод `select_one` выберёт первый элемент, который попадёт под ваше описание. Давайте проверим, что у нас на странице ровно 20 элементов с классом instock. Для этого в поиске пишем точкаi nstock. Да, всё верно, элементов ровно 20 В этом случае используем второй способ получения статуса о наличии.

In [ ]:
instock = book.select_one('div[class=product_price] p[class*=instock]').text
print(instock)
print(instock.strip())

Вы видите, что мы указали и родительский элемент. А для тега `<p>` — класс со звёздочкой, так как на странице у него есть ещё один класс — `availability`. Вообще, указывать точный путь до элементов довольно нудно и долго, но иногда только так вы можете получить нужную информацию.

Теперь попробуем получить цену. Самое простое решение — найти тег `<p>` с классом `price_color`, но лучше разобрать и другие методы BeautifulSoup. Так что давайте воспользуемся методом `find_previous_sibling()`, который позволяет искать предыдущие теги одного уровня, а `find_next_sibling()`
ищет следующие теги одного уровня. Внутри мы указываем, какой тег хотим найти. Это удобно в тех случаях, когда много одинаковых тегов, а классы ничем не отличаются. Если вы добавите `s` в конце — `find_next_siblings()` и `find_previous_siblings()` — то вам вернётся список. Итак, ищем цену. Копируем уже найденную сущность `instock`, убираем получение текста и добавляем `find_previuos_sibling()`.
Внутри указываем тег `<p>`. Смотрим, что получилось.

In [ ]:
instock = book.select_one('div[class=product_price] p[class*=instock]')
price = instock.find_previous_sibling(‘p')
print(price)

Как видите, мы нашли цену. Осталось найти изображение обложки. Оно лежит в теге `<img>`, вложенном в тег `<а>`, родителем которого является тег `<div>` с классом `image_container`. У тега `<img>` нам надо получить
атрибут `src`. Давайте всё это запишем.

In [ ]:
image = soup.find('div', attrs={'class': 'image_container'}).find('img')['src']
print(image)

Обратите внимание, что ссылка у нас неполная. Желательно сохранять полные ссылки, чтобы вы могли воспользоваться ими без необходимости открывать сайт и вспоминать, как именно выглядит ссылка. Чтобы понять, какая ссылка полная, можно кликнуть на картинку и посмотреть, что будет в адресной строке. Копируем недостающую часть. Теперь сохраним переменную `image_link` таким образом.

In [ ]:
image_link = ‘https://books.toscrape.com/’+image
print(image_link)

Теперь у нас есть полная ссылка на изображение обложки.

Последнее, что нам осталось сделать, чтобы спарсить все книги с первой страницы сайта, — создать цикл, в котором мы будем идти по каждой книге, извлекать название, цену, обложку и наличие, сохранять всё это в словарь, а словарь сохранять в список. Таким образом, в итоге у нас должен
получиться список из 20 книг с первой страницы сайта.

Давайте объединим все участки кода, которые мы уже написали и добавим совсем немного.

In [ ]:
books_list = []

for book in books:
    title = book.find('h3').find('a')['title']
    instock = book.select_one('div[class=product_price] p[class*=instock]').text.strip()
    price = book.select_one('div[class=product_price] p[class*=instock]').find_previous_sibling('p').text.strip()
    image = soup.find('div', attrs={'class': 'image_container'}).find('img')['src']
    image_link = 'https://books.toscrape.com/'+image
    book_dict = {
        'Image': image_link,
        'Title': title,
        'Price': price,
        'Instock': instock
        }
    
books_list.append(book_dict)
print(len(books_list))
pprint(books_list)

В нашем итоговом списке 20 книг, как и должно быть. При просмотре этого списка мы видим, что он состоит из 20 словарей, в каждом из которых находится вся нужная информация о книге. 

Так же в Beautiful Soup есть много других полезных методов: например, `find_parent()`, `find_next()` и
`find_previous()`, которые помогают найти родительский тег, следующий или предыдущий тег соответственно. При работе с любыми библиотеками полезно пользоваться их документацией или искать ответы на вопросы на сайте StackOverflow по тегу BeautifulSoup.

### Глоссарий
- `HTML (HyperText Markup Language)` — язык гипертекстовой разметки. Интерпретируется браузерами, в результате чего форматированный текст отображается на экране. HTML-страницы передаются от сервера к клиенту по протоколу HTTP в виде обычного или зашифрованного текста.

- `HTML-теги` — используются для разграничения начала и конца элементов в разметке.

- `Атрибуты` — свойства тега, дающие дополнительные возможности форматирования текста.

- `DOM (Document Object Model)` — не зависящий от платформы и языка программный интерфейс, позволяющий программам и скриптам получить доступ к содержимому HTML-, XHTML- и XML- документов, а также изменять содержимое, структуру и оформление таких документов.

- `Библиотека Beautiful Soup` — парсер lxml, который преобразует наш HTML-код в DOM и обрабатывает полученную структуру.

- `Дерево синтаксического разбора` — структуры данных Beautiful Soup, которые создаются по мересинтаксического разбора документа.

### Дополнительные материалы
1. Документация BeautifulSoup 

### Используемая литература
1. Базовая структура HTML .
2. Элементы HTML/CSS .
3. Объектная модель документа .
4. Стандарт HTML5 .
5. Документация Beautifull Soup .
6. Документации/BeautifulSoup .